In [ ]:
import plotly
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.patches as mpatches
import matplotlib.colors as mplc
import re
import plotly.colors
from plotly.offline import plot
from plotly.subplots import make_subplots
# !pip install repo2data
# from repo2data.repo2data import Repo2Data
!git clone https://github.com/evaalonsoortiz/QC-imaging-demographics.git

# Introduction

 The purpose of this Jupyter Notebook is to explore demographic data gathered from human MRI and/or PET brain imaging studies conducted in Quebec, Canada from 1992 to 2023. We would like to visualize trends and relationships between various demographic factors, as well as assess whether there are issues in participant selection and in the reporting of demographic data.

 **Why do we focus on demographics**
 Our understanding of human health comes from scientific studies, each typically focusing on a specific aspect of disease and treatment. These studies shape medical research priorities and clinical decision-making. However, if a study's participant sample is not representative of the broader population, its findings may only benefit certain groups. For example, if a landmark Alzheimer's study primarily includes male participants but makes general conclusions about disease progression, it risks creating a misleading narrative, one that might differ if the study included more female participants or a balanced sampe.

 **What is the potential problem with reportng?**
 Standards for reporting demographic data vary across publications and institutions, and many reputable journals may not prioritize details like sex, race and ethnicity, or age. When this information is omitted, readers may struggle to determine which populations the study’s conclusions accurately appy to.

 **The content of this Jupytr book**
 In this notebook, you will find figures illustrating the demographics of Quebec MRI and PET brain imaging study participants, some of which contain interactive elements, and brief descriptions of what has been done to create those figures and why. Discussion and interpretation of results can be found in the full preprint.

# Study selection
We used Medline, Embase, and Google Scholar databases, retrieved all records involving MRI and/or PET brain imaging in Quebec between 1992 and 2023, and screened those results. The Sankey diagram shows the screening process in detail. You can hover over the connections between blocks for information regarding screening criteria.

## Figure 1

In [ ]:
screening_info = ['Records obtained from Embase',
                  'Records obtained from MEDLINE',
                  'Records obtained from Google Scholar',
                  'Records selected for screening',
                  'Duplicate references removed',
                  """
                    Title and abstract screened for relevance:<br>
                    - Studies not concerning brain imaging<br>
                    - Studies not concerning human populations<br>
                    - Studies not concerning MRI or PET<br>
                    - Studies not concerning Quebec population
                  """,
                  'Studies retrieved for evaluation',
                  """
                    Exclusion criteria:<br>
                    - Review articles (n=31);<br>
                    - Ambiguous CT or MRI patient counts (n=14);<br>
                    - Case Reports (n=99);<br>
                    - Animal studies (n=14);<br>
                    - Correspondence (n=1);<br>
                    - Public dataset (n=84);<br>
                    - Conference paper (n=16);<br>
                    - Multisite dataset with unknown participant counts (n=776);<br>
                    - Post-mortem brain (n=14);<br>
                    - Registered report (n=2);<br>
                    - Wrong study design (n=8);<br>
                    - Unspecified location (n=37);<br>
                    - Study population outside Quebec (n=2052);<br>
                    - Duplicate dataset (n=104);
                  """,
                  'Studies selected for analysis']

fig1 = go.Figure(data=[go.Sankey(
    arrangement = "freeform",
    node = dict(
      pad = 80,
      thickness = 10,
      line = dict(color = "black", width = 0.5),
      label = ["References obtained from Embase",#0
               "References obtained from MEDLINE",#1
               "References obtained from Google Scholar",#2
               "Main references identified",#3
               "Studies screened",#4
               "References removed",#5
               "Studies excluded from retrieval", #6
               "Studies assessed for eligibility",#7
               "Studies exluded from review",#8
               "Studies included in review"],#9
      x = [0, 0, 0, 0.3, 0.5, 0.5, 0.7, 0.7, 0.9, 0.9],
      y = [2, 0, 0, 0.5, 0.3, 0.9, 0.8, 0.4, 0.6, 0.2],
      hovertemplate = "%{label}<extra>%{value}</extra>",
      color = ["darkblue","darkblue","darkblue","darkblue","darkgreen","darkred","darkred","darkgreen","darkred","darkgreen"]
    ),
    link = dict(
      source = [0, 1, 2, 3, 3, 4, 4, 7, 7],
      target = [3, 3, 3, 4, 5, 6, 7, 8, 9],
      value = [6429, 3816, 100, 7279, 3068, 2476, 4801, 3252, 1549],
      customdata = screening_info,
      hovertemplate = "%{customdata}",
  ))])

fig1.update_layout(title = dict(text="Study identification methodology"),
                   width=800,
                   height=500,
                   font_size=11,
                   margin=dict(l=0))
    
fig1.show()

In [ ]:
# load data
# data_req = os.path.join("..", "..", "binder", "data_requirement.json")
# repo2data=Repo2Data(data_req)
# data_path = repo2data.install()[0]
data_path = os.path.join("QC-imaging-demographics", "data")
# fname_data = os.path.join(path_data, f"fcdata.xlsx")
# df = pd.read_excel(fname_data)
df = pd.read_excel(os.path.join(data_path, 'fcdata.xlsx'))

# load full dataset
# will need this for treemap with DOIs
# fname2 = os.path.join(path_data, f"Cleaned Full data.xlsx")
df_full = pd.read_excel(os.path.join(data_path, 'Cleaned Full data.xlsx'), sheet_name='Full that matches the formatted')

# merge superfluous ethnicity columns
df['Black'] = df.loc[:,['Black','African-American']].sum(axis=1)
df['White'] = df.loc[:,['Caucasian','Caucasian-Hispanic']].sum(axis=1)
df['Asian'] = df.loc[:,['Asian','Asian American']].sum(axis=1)
df = df.drop(columns=['African-American','Caucasian','Caucasian-Hispanic','Asian American'])
df.fillna({'Other': 0, 'Not specified' : 0, 'Middle Eastern' : 0, 'Caribbean' : 0, 'Jewish' : 0}, inplace=True)

###############
# create total participant column on the rule that if PET=MRI then all participants were scanned in both, and if not then total = PET+MRI
df['Total participants'] = df.loc[:,['PET participants','MRI participants']].sum(axis=1)
df.loc[df['PET participants']==df['MRI participants'], 'Total participants'] = df['MRI participants']

# hard coding execptions to the above rule
df.loc[118, 'Total participants'] = 25
df.loc[268, 'Total participants'] = 61
df.loc[356, 'Total participants'] = 15
df.loc[477, 'Total participants'] = 29
df.loc[572, 'Total participants'] = 24
df.loc[626, 'Total participants'] = 20
df.loc[832, 'Total participants'] = 60
df.loc[840, 'Total participants'] = 21
df.loc[1113, 'Total participants'] = 34
df.loc[1128, 'Total participants'] = 35
df.loc[1143, 'Total participants'] = 14
df.loc[1178, 'Total participants'] = 9
df.loc[1215, 'Total participants'] = 40
df.loc[1293, 'Total participants'] = 262
df.loc[1325, 'Total participants'] = 31
df.loc[1369, 'Total participants'] = 44
df.loc[1446, 'Total participants'] = 54
df.loc[1488, 'Total participants'] = 39
###############

df_full['MRI participants'] = pd.to_numeric(df_full['MRI participants'], errors='coerce').fillna(0)
df_full['PET participants'] = pd.to_numeric(df_full['PET participants'], errors='coerce').fillna(0)
df_full['Male'] = pd.to_numeric(df_full['Male'], errors='coerce').fillna(0)
df_full['Female'] = pd.to_numeric(df_full['Female'], errors='coerce').fillna(0)
df_full['Total participants'] = df_full.loc[:,['PET participants','MRI participants']].sum(axis=1)
df_full.loc[df_full['PET participants']==df_full['MRI participants'], 'Total participants'] = df_full['MRI participants']
df_full.loc[17, 'Total participants'] = 25
df_full.loc[1077, 'Total participants'] = 61
df_full.loc[833, 'Total participants'] = 15
df_full.loc[870, 'Total participants'] = 29
df_full.loc[667, 'Total participants'] = 24
df_full.loc[498, 'Total participants'] = 20
df_full.loc[757, 'Total participants'] = 60
df_full.loc[342, 'Total participants'] = 21
df_full.loc[1548, 'Total participants'] = 34
df_full.loc[41, 'Total participants'] = 35
df_full.loc[1128, 'Total participants'] = 14
df_full.loc[517, 'Total participants'] = 9
df_full.loc[987, 'Total participants'] = 40
df_full.loc[1499, 'Total participants'] = 262
df_full.loc[82, 'Total participants'] = 31
df_full.loc[1061, 'Total participants'] = 44
df_full.loc[591, 'Total participants'] = 54
df_full.loc[42, 'Total participants'] = 39


# Create unreported column based on total participants
df['Unreported Ethnicity'] = df.loc[:,'Total participants'] - df.loc[:,['Black','Asian','Other','Not specified','Middle Eastern','Caribbean','Jewish','White']].sum(axis=1)
i = list(df.columns)
a, b = i.index('Unreported Ethnicity'), i.index('Total participants')
i[b], i[a] = i[a], i[b]
df = df[i]


# this is our initial dataframe which will be manipulated individually for each analysis to account for needing to separate different studies based on other criteria
# e.g. needing to remove multiple region studies for region analysis, but leaving those in for sex/ethnicity/age analysis

In [ ]:
# Categorize pathologies

df_paths = df_full.copy()

# categorizing patient populations
pathology = {"Epilepsy" : "epilepsy|epileptic",
             "Parkinson's" : "Parkinson",
             "Alzheimer's" : "Alzheimer",
             "Mental and behavioral disorders" : "Schizophrenia|depression|depressive|anxiety|psychosis|addiction|autism|alcohol|amusia|serotonin|emotion|PTSD|suicide|suicidal|substance|Attention|bipolar|OCD|personality",               
             "Multiple Sclerosis" : "Multiple Sclerosis",                 
             "Cerebrovascular disease" : "Carotid|Stenosis|Stroke|vascualar",                 
             "Cancer and tumors" : "tumor|cancer|adenoma|glioblastoma|angioma|neurofibromatosis|glioma|lymphoma|meningioma|schwannoma|hamartoma|sarcoma|leukemia",                 
             "Newborns and infants" : "Neonatal|infant|newborn|child",
             "Concussion and Injury" : "Injury|Concussion",
             "Vision disorders" : "yopia|blind|hemianopia",
             "Pain" : "pain", 
             "Healthy population" : "Not specified|Healthy|aging|volunteer"}

# make categorized pathology columns based on disease characteristics keywords
for p, keywords in pathology.items():
    df_paths[p] = df_paths['Disease Characteristics'].str.contains(keywords, flags=re.IGNORECASE, regex=True)

df_paths['number of pathologies'] = df_paths[df_paths.columns[23:35]].sum(axis=1)
df_paths['Pathology'] = ""

# run through individual pathology columns and set the pathology for each true value
for n in df_paths.columns[23:35]:
    df_paths.loc[df_paths[n]==True,'Pathology'] = n

# categorize anything with multiple pathologies or anything lying outside the scope of our keywords
for i in df_paths.index:
    if df_paths.loc[i,'number of pathologies'] == 0:
        df_paths.loc[i,'Pathology'] = "Other"
    elif df_paths.loc[i,'number of pathologies'] > 1:
        df_paths.loc[i,'Pathology'] = "Multiple categories"

# fix participant columns 
df_paths['MRI participants'] = pd.to_numeric(df_paths['MRI participants'], errors='coerce')
df_paths['PET participants'] = pd.to_numeric(df_paths['PET participants'], errors='coerce')

# drop two studies with ambiguous mri/pet participant counts (no values listed)
i = df_paths.loc[df_paths['MRI participants'].isna() & df_paths['PET participants'].isna()].index
df_paths = df_paths.drop(i)

# catgeorize mri/pet/both
df_paths['MRI or PET'] = ""
for i in df_paths.index:
    if (df_paths['MRI participants'][i]==0):
        df_paths.loc[i,'MRI or PET'] = 'PET'
    elif (df_paths['PET participants'][i]==0):
        df_paths.loc[i,'MRI or PET'] = 'MRI'
    else:
        df_paths.loc[i,'MRI or PET'] = 'MRI and PET'

# handle studies with multiple geographical locations
idx = df_paths['Geographical location'].str.contains(',')
df_paths.loc[idx,'Geographical location'] = 'Multiple regions'

We identified 1549 studies that were conducted in Quebec using MRI, PET, or both imaging modalities to study the healthy or diseased brain. These studies took place in different parts of Quebec, and focused on a wide variety of conditions. A breakdown of how studies were categorized is shown in the following treemap. 

You can click on each box in the figure to expand the category, and each individual study can be expanded to see the title and a DOI link to the article. You may click at the top of the current box to zoom back out.
 
The first level of distinction is the imaging technique. The second level is the region(s) of Quebec in which the studies take place. The final level is a categorization of study populations among the following: healthy population (studies of healthy brain connectivity and cognition, testing of imaging techniques in healthy populations, etc.), epilepsy, Parkinson's disease, Alzheimer's disease, mental and behavioral disorders (excluding dementia), multiple sclerosis, cerebrovascular disease, cancer and tumors, newborns and infants, concussion and injury, vision disorders, and pain. Categories also exist for populations concerning multiple of the above categories, labelled “Multiple Categories”, and for studies concerning any other patient population which has not been categorized here, labelled “Other”.

## Figure 2

In [ ]:
year_str = df_paths['Year of Publication'].astype(str)
df_paths['Geographical location'] = df_paths['Geographical location'].replace("Montreal\n",'Montreal')

# list study info line as (first author et al., year)
df_paths['Study info'] = df_paths['Authors'].str.split(',').str[0].str.split().str[-1] + ' et al., ' + year_str

# construct hyperlink for each paper's unique DOI
df_paths['Link'] = df_paths['DOI']
df_paths['Link'] = df_paths['Link'].replace('http',"""<a style='color:white' href='http""", regex=True)
df_paths['Link'] = df_paths['Link'] + """'>->Go to the paper</a>"""

# make text box for each entry with the link and title
df_paths['Summary'] = df_paths['Link'] + '<br><br>Title: ' + df_paths['Title'].astype(str)
df_paths['N'] = np.ones((len(df_paths),1))

# handle duplicates of study info (multiple studies with same year and first author last name)
for i in range(6):
    dupes = df_paths['Study info'].duplicated()
    df_paths.loc[dupes,'Study info'] = df_paths.loc[dupes,'Study info'] + ' '

df_paths = df_paths.sort_values('Study info')

# construct dataframe for the treemap from the list of studies
args = dict(data_frame=df_paths, values='N',
            color='N', hover_data='Study info',
            path=['MRI or PET', 'Geographical location', 'Pathology', 'Study info'])
args = px._core.build_dataframe(args, go.Treemap)
treemap_df = px._core.process_dataframe_hierarchy(args)['data_frame']

fig2 = go.Figure(go.Treemap(
    ids=treemap_df['id'].tolist(),
    labels=treemap_df['labels'].tolist(),
    parents=treemap_df['parent'].tolist(),
    values=treemap_df['N'].tolist(),
    branchvalues='remainder',
    text=df_paths['Summary'],
    hoverinfo='label', # Set to None if sorting by DOI
    # customdata = treemap_df['Study info'], # use if sorting by DOI
    # hovertemplate = "<b>%{customdata}</b><extra></extra>", # to be used if sorting by DOI
    textfont=dict(
        size=15,
    )
))

fig2 = fig2.update_layout(
    autosize=False,
    width=1000,
    height=650,
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=45,
    )
)

fig2.show()

In [ ]:
# Categorize geographically by number of studies 
# Drop studies with multiple locations, plot number of studies by city
filtered = df[~df['Geographical location'].str.contains(',')]
# removed 10 studies with multiple locations listed

In [ ]:
#counting studies by region
# all
s1 = filtered['Geographical location'].value_counts()
# PET only
s2 = filtered[filtered['PET participants'].notna() & filtered['MRI participants'].isna()]['Geographical location'].value_counts()
# MRI only
s3 = filtered[filtered['PET participants'].isna() & filtered['MRI participants'].notna()]['Geographical location'].value_counts()
# PET+MRI studies
s4 = filtered[filtered['PET participants'].notna() & filtered['MRI participants'].notna()]['Geographical location'].value_counts()

# all
regs_all = pd.DataFrame({'Region' : s1.index, 'count' : s1.values})
regs_all['Region'] = ['Montreal', 'Estrie', 'Capitale-Nationale', 'Mauricie', 'Saguenay - Lac-Saint-Jean', 'Monteregie']
regs_all.loc[len(regs_all.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_all.loc[len(regs_all.index)]=['Abitibi-Temiscamingue', 0]
regs_all.loc[len(regs_all.index)]=['Outaouais', 0]
regs_all.loc[len(regs_all.index)]=['Nord-du-Quebec', 0]
regs_all.loc[len(regs_all.index)]=['Laurentides', 0]
regs_all.loc[len(regs_all.index)]=['Lanaudiere', 0]
regs_all.loc[len(regs_all.index)]=['Chaudiere-Appalaches', 0]
regs_all.loc[len(regs_all.index)]=['Cote-Nord', 0]
regs_all.loc[len(regs_all.index)]=['Bas-Saint-Laurent', 0]
regs_all.loc[len(regs_all.index)]=['Centre-du-Quebec', 0]
regs_all.loc[len(regs_all.index)]=['Laval', 0]

# PET only
regs_pet = pd.DataFrame({'Region' : s2.index, 'count' : s2.values})
regs_pet['Region'] = ['Montreal', 'Capitale-Nationale', 'Estrie']
regs_pet.loc[len(regs_pet.index)]=['Mauricie', 0]
regs_pet.loc[len(regs_pet.index)]=['Saguenay - Lac-Saint-Jean', 0]
regs_pet.loc[len(regs_pet.index)]=['Monteregie', 0]
regs_pet.loc[len(regs_pet.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_pet.loc[len(regs_pet.index)]=['Abitibi-Temiscamingue', 0]
regs_pet.loc[len(regs_pet.index)]=['Outaouais', 0]
regs_pet.loc[len(regs_pet.index)]=['Nord-du-Quebec', 0]
regs_pet.loc[len(regs_pet.index)]=['Laurentides', 0]
regs_pet.loc[len(regs_pet.index)]=['Lanaudiere', 0]
regs_pet.loc[len(regs_pet.index)]=['Chaudiere-Appalaches', 0]
regs_pet.loc[len(regs_pet.index)]=['Cote-Nord', 0]
regs_pet.loc[len(regs_pet.index)]=['Bas-Saint-Laurent', 0]
regs_pet.loc[len(regs_pet.index)]=['Centre-du-Quebec', 0]
regs_pet.loc[len(regs_pet.index)]=['Laval', 0]

# MRI only
regs_mri = pd.DataFrame({'Region' : s3.index, 'count' : s3.values})
regs_mri['Region'] = ['Montreal', 'Estrie', 'Capitale-Nationale', 'Mauricie', 'Saguenay - Lac-Saint-Jean', 'Monteregie']
regs_mri.loc[len(regs_mri.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_mri.loc[len(regs_mri.index)]=['Abitibi-Temiscamingue', 0]
regs_mri.loc[len(regs_mri.index)]=['Outaouais', 0]
regs_mri.loc[len(regs_mri.index)]=['Nord-du-Quebec', 0]
regs_mri.loc[len(regs_mri.index)]=['Laurentides', 0]
regs_mri.loc[len(regs_mri.index)]=['Lanaudiere', 0]
regs_mri.loc[len(regs_mri.index)]=['Chaudiere-Appalaches', 0]
regs_mri.loc[len(regs_mri.index)]=['Cote-Nord', 0]
regs_mri.loc[len(regs_mri.index)]=['Bas-Saint-Laurent', 0]
regs_mri.loc[len(regs_mri.index)]=['Centre-du-Quebec', 0]
regs_mri.loc[len(regs_mri.index)]=['Laval', 0]

# Studies with MRI and PET imaging
regs_both = pd.DataFrame({'Region' : s4.index, 'count' : s4.values})
regs_both['Region'] = ['Montreal', 'Estrie', 'Capitale-Nationale']
regs_both.loc[len(regs_both.index)]=['Mauricie', 0]
regs_both.loc[len(regs_both.index)]=['Saguenay - Lac-Saint-Jean', 0]
regs_both.loc[len(regs_both.index)]=['Monteregie', 0]
regs_both.loc[len(regs_both.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_both.loc[len(regs_both.index)]=['Abitibi-Temiscamingue', 0]
regs_both.loc[len(regs_both.index)]=['Outaouais', 0]
regs_both.loc[len(regs_both.index)]=['Nord-du-Quebec', 0]
regs_both.loc[len(regs_both.index)]=['Laurentides', 0]
regs_both.loc[len(regs_both.index)]=['Lanaudiere', 0]
regs_both.loc[len(regs_both.index)]=['Chaudiere-Appalaches', 0]
regs_both.loc[len(regs_both.index)]=['Cote-Nord', 0]
regs_both.loc[len(regs_both.index)]=['Bas-Saint-Laurent', 0]
regs_both.loc[len(regs_both.index)]=['Centre-du-Quebec', 0]
regs_both.loc[len(regs_both.index)]=['Laval', 0]

In [ ]:
# Load in quebec administrative region data
# from: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/quebec.geojson
# had to edit labels to remove accents so they could be read
geoname_data = os.path.join(data_path, "quebec.geojson")

with open(geoname_data) as f:
    var_geojson = json.load(f)

# Regional breakdown of studies

The studies within Quebec were conducted in different administrative regions of the province. The following map shows the regional distribution of studies that were conducted with MRI and/or PET. You may zoom and drag the map. The drop down menu can be used to change the imaging modality, and each bubble can be hovered over to see the corresponding region’s name and number of studies.
 
## Figure 3

In [ ]:
colorscale = [[0.0, "#FFFFC0"], [0.5, "#CBC7C0"], [1.0, "#FFFFF0"]]

# filled grey map of administrative regions
trace1 = go.Choropleth(geojson=var_geojson,
                      showscale=False,
                      colorscale=colorscale,
                      zmin=0, zmax=1,
                      z=[0.5]*len(var_geojson['features']),
                      locations=regs_all['Region'],
                      featureidkey='properties.name',
                      hoverinfo='skip')


# sized markers for number of studies by region
maxval=regs_all['count'].max()
minval=regs_all['count'].min()
trace2 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_all['count'],0.5)*10, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_all['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_all['Region'],
                       featureidkey='properties.name',
                       customdata=regs_all[['Region','count']],
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

maxval=regs_pet['count'].max()
minval=regs_pet['count'].min()
trace3 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_pet['count'],0.5)*30, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_pet['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_pet['Region'],
                       featureidkey='properties.name',
                       customdata=regs_pet[['Region','count']],
                       visible=False,
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

maxval=regs_mri['count'].max()
minval=regs_mri['count'].min()
trace4 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_mri['count'],0.5)*10, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_mri['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_mri['Region'],
                       featureidkey='properties.name',
                       customdata=regs_mri[['Region','count']],
                       visible=False,
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

maxval=regs_both['count'].max()
minval=regs_both['count'].min()
trace5 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_both['count'],0.5)*15, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_both['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_both['Region'],
                       featureidkey='properties.name',
                       customdata=regs_both[['Region','count']],
                       visible=False,
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

# remove all default layout elements, only interested in quebec
layout = go.Layout(
    geo=dict(showland=False,
             showcountries=False,
             showocean=False,
             showrivers=False,
             showlakes=False,
             showcoastlines=False),
    title='Distribution of all PET and MRI studies in Quebec')

fig = go.Figure(data = [trace1, trace2, trace3, trace4, trace5], layout=layout)
#conic conformal projection recommended by Statistics Canada https://www150.statcan.gc.ca/n1/pub/92-195-x/2011001/other-autre/mapproj-projcarte/m-c-eng.htm
fig.update_geos(fitbounds="locations", projection_type='conic conformal') 
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="All studies",
                     method="update",
                     args=[{"visible": [True, True, False, False, False]},
                           {"title": "Distribution of all PET and MRI studies in Quebec"}]),
                dict(label="PET studies",
                     method="update",
                     args=[{"visible": [True, False, True, False, False]},
                           {"title": "Distribution of PET studies in Quebec"}]),
                dict(label="MRI studies",
                     method="update",
                     args=[{"visible": [True, False, False, True, False]},
                           {"title": "Distribution of MRI studies in Quebec"}]),
                dict(label="PET+MRI studies",
                     method="update",
                     args=[{"visible": [True, False, False, False, True]},
                           {"title": "Distribution of studies in Quebec scanning participants in both PET and MRI"}]),
            ]),
        )
    ])

fig.update_layout(width=900, height=700)
fig.show()


# important to note, this is by region but even outside of montreal these are mostly in cities.
# the 46 in Capitale-Nationale are all in Quebec City. Of the 53 in Estrie, 52 are specified as Sherbrooke. 
# Mauricie = Trois-Rivieres, monteregie = Longueil


# Basic demographic breakdown

We aimed to visualize how imaging studies are distributed across age groups to understand which populations are most represented and why certain age groups are commonly studied. To achieve this, we categorized studies based on the average age of the participants and plotted the results in a histogram. 

On the right, in gray, are studies that failed to report participant ages or only provided limited information, such as an age range. Using the drop down menu, you can choose whether the histogram displays the number of studies within each age bin or the total number of participants in those studies.
 
Both counting methods are similarly limited. Binning by average age removes details about the age distribution within each study, which may lead to a misleading representation. However, since most studies do not provide comprehensive demographic breakdowns, this visualization offers the best possible insight given the available data.

## Figure 4

In [ ]:
# Age analysis
# Histogram of average age per study
# rename for simplicity
df = df.rename(columns = dict({'Average Age (Years)' : 'Age'}))

noagedf = df[df['Age'].isna()]
# studies with no age reported.

# drop all nans and nonnumerics (entries with > or other symbols)
Agedf = df.copy()
Agedf['Age'] = pd.to_numeric(Agedf['Age'], errors='coerce')
Agedf = Agedf.dropna(subset=['Age'])


bins = np.arange(0,90,5)

agebincounts = []
agecounts = []
labels = []
for i,x in enumerate(bins):
    agebincounts.append( len(Agedf[(Agedf['Age'] >= x) & (Agedf['Age'] < (x+5))].index) )
    agecounts.append(Agedf.loc[(Agedf['Age'] >= x) & (Agedf['Age'] < (x+5)), 'Total participants'].sum())
    labels.append( f'{x}-{x+5}' )

unreported_studies = len(noagedf.index)
unreported_participants = noagedf['Total participants'].sum()

ages1 = pd.DataFrame({'Range':labels, 'count':agebincounts})
ages2 = pd.DataFrame({'Range':labels, 'count':agecounts})

ages1.loc[len(ages1.index)] = ['Unreported', unreported_studies]
ages2.loc[len(ages2.index)] = ['Unreported', unreported_participants]

colors = ["royalblue"]*18 + ["dimgrey"]

bar1 = go.Bar(
    x = ages1['Range'],
    y = ages1['count'],
    marker_color=colors,
    customdata=ages1['count']/ages1['count'].sum()*100,
    hovertemplate="Percent of studies: %{customdata:.3f}%<extra></extra>",
    yaxis='y'
)

bar2 = go.Bar(
    x=ages2['Range'],
    y=ages2['count'],
    marker_color=colors,
    customdata=ages2['count']/ages2['count'].sum()*100,
    hovertemplate="Percent of participants: %{customdata:.3f}%<extra></extra>",
    visible=False,
    yaxis='y'
)

bar1p = go.Bar(
    x = ages1['Range'],
    y = ages1['count']/ages1['count'].sum()*100,
    marker_color=colors,
    hovertemplate="Percent of studies: %{value:.3f}%<extra></extra>",
    yaxis='y2',
    visible=True
)

bar2p = go.Bar(
    x = ages2['Range'],
    y = ages2['count']/ages2['count'].sum()*100,
    marker_color=colors,
    hovertemplate="Percent of participants: %{value:.3f}%<extra></extra>",
    yaxis='y2',
    visible=False
)

fig=go.Figure(data=[bar1,bar2,bar1p,bar2p],
             layout={
                 'yaxis' : {'title':'Number of studies', 'showgrid':False},
                 'yaxis2' : {'title': 'Percent of studies', 'overlaying': 'y', 'side': 'right'}
             })

fig.update_layout(title='Studies by average age of participants', xaxis_tickangle=-45,
                 barmode='group', width=1100, height=650, showlegend=False, plot_bgcolor='rgba(0.2, 0.2, 0.2, 0.2)',
                 xaxis=dict(title="Average age"),
                 margin=dict(t=140, b=0, l=0, r=0))

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="By number of studies",
                     method="update",
                     args=[{"visible": [True, False, True, False]},
                           {"title": "Studies by average age of participants",
                            'yaxis' : {'title':'Number of studies', 'showgrid':False},
                            'yaxis2' : {'title': 'Percent of studies', 'overlaying': 'y', 'side': 'right'}}]),
                dict(label="By number of participants",
                     method="update",
                     args=[{"visible": [False, True, False, True]},
                           {"title": "Participant totals from studies by average age",
                            'yaxis' : {'title':'Number of participants', 'showgrid':False},
                            'yaxis2' : {'title': 'Percent of participants', 'overlaying': 'y', 'side': 'right'}}])
            ]),
            xanchor='left',
            y=1.12,
            pad={"l": 50, "t": 0}
        )]
    )

We also wanted to see if studies were properly reporting participant sex, and if they were appropriately including male and female participants. The split of male and female participants as well as participants whose sex is unreported is shown in this pie chart.

## Figure 5

In [ ]:
# Sex analysis 

# male and female totals reported in most studies
male = df_full['Male'].sum()
female = df_full['Female'].sum()
# majority of unreported sex is studies which report no sex at all, however some is from studies which only partially report sex
unreported = (df_full.loc[:,'Total participants'] - df_full.loc[:,['Male','Female']].sum(axis=1)).sum()
# unreported = df.loc[(df['Male']==0) & (df['Female']==0),'Total participants'].sum() 
# if you assumed all reporting studies have full information lose 348 participants with no sex in studies that report some sex
plt.style.use('ggplot')
sns.set_palette("colorblind")
labels = ['Male', 'Female', 'Unreported']
sizes = [male, female, unreported]
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels,autopct='%1.1f%%',explode=[0,0,.15])
plt.title(f'Breakdown of total participants by sex (n = {int(male + female + unreported)})')
plt.show()

# Relation between age and sex

Our initial approach to analyzing age and sex demographics in imaging studies was to use a population pyramid, where age distributions for each sex are displayed as stacked horizontal histograms. However, this method presents challenges due to inconsistencies in how studies report age.

Some studies provide detailed age information, either by reporting individual ages (which is rare) or by reporting distribution metrics such as standard deviation alongside the mean. However, most studies lack this level of detail. By grouping studies solely based only on average age, we lose crucial information about the actual age distribution within each study. Additionally, this approach assumes that the age distributions of male and female participants within a study are similar, which may not be the case, potentially further skewing the data.
## Figure 6


In [ ]:
# Age pyramid

# drop all rows with no age information, and eliminate nonnumerics (e.g. entries where age is listed as >0.69)
Agedf2 = df[df['Age'].notna()].copy()
Agedf2['Age'] = pd.to_numeric(Agedf2['Age'], errors='coerce')
Agedf2 = Agedf2.dropna(subset=['Age'])

# drop rows with no male or female information
Agedf2 = Agedf2[~((Agedf2['Female'].isna()) & Agedf2['Male'].isna())]
# Here we are down to 1085 studies from 1553

bins = np.arange(0,90,5)
agecounts_male = []
agecounts_female = []
labels = []
for i,x in enumerate(bins):
    agecounts_male.append( Agedf2.loc[(Agedf2['Age'] >= x) & (Agedf2['Age'] < (x+5)), 'Male'].sum() )
    agecounts_female.append( Agedf2.loc[(Agedf2['Age'] >= x) & (Agedf2['Age'] < (x+5)), 'Female'].sum() )
    labels.append( f'{x}-{x+5}' )


labels.reverse()
agecounts_male.reverse()
agecounts_female.reverse()
pyramid_df = pd.DataFrame(data={'age range':labels, 'male':agecounts_male, 'female':agecounts_female})
pyramid_df['male'] = pyramid_df['male'] * -1

# plt.grid()
ax1 = sns.barplot(x='male', y='age range', data=pyramid_df, palette="Reds", hue=labels)
ax2 = sns.barplot(x='female', y='age range', data=pyramid_df, palette="Blues", hue=labels)
plt.title("Population pyramid for PET and MRI study participants in Quebec")
plt.xlabel("             Male    Female")
plt.xticks(ticks=[-4000, -2000, 0, 2000, 4000], labels=[4000, 2000, 0, 2000, 4000])

plt.show()

To get a better idea on how the sex and age of participants might be related, a relational scatter plot was created. This plot compares the percentage of female and male participants vs. the average age of participants for each study. Each marker represents an individual study. Markers are colored based on the number of participants. Because a large number of low sample size studies have poor sex distribution, it is worth taking a look at the graph when looking at only studies with a larger sample size. Here, the cutoff for the sample size can be chosen with a slider. Each marker can be hovered over to see the sex ratio, mean age, and number of participants. 

Ultimately, both visualizations suffer from the loss of age distribution information within each study, highlighting the limitations of current age reporting practices. This reinforces the need for more comprehensive demographic reporting to enable more accurate and meaningful analyses.


## Figure 7

In [ ]:
# relational age/sex plot with percentages
# make percentage column 
df['Male'] = df['Male'].fillna(0)
df['Female'] = df['Female'].fillna(0)
ser = df['Female'] / df.loc[:,['Female','Male']].sum(axis=1) *100
try:
    df.insert(7, "Percentage female", ser)
except ValueError:
    pass

# Joint distribution of average age of participants and sex as a percentage of participants
df_rel = df[~(df['Percentage female'].isnull())].copy()
df_rel['Age'] = pd.to_numeric(df_rel['Age'], errors='coerce')
df_rel = df_rel.dropna(subset=['Age'])

norm = mplc.LogNorm(vmin=10, vmax=300, clip=True)

fig = make_subplots(1,1,horizontal_spacing=0,specs=[[{"secondary_y": True}]])

for step in np.arange(1, 51, 1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            name=str(step),
            mode='markers',
            customdata=df_rel.loc[df_rel['Total participants']>=step,'Total participants'],
            hovertemplate="Mean age: %{x}<br>Percent female: %{y:.2f}%<br>Number of participants: %{customdata}<extra></extra>",
            marker=dict(
                color=norm(df_rel.loc[df_rel['Total participants']>=step,'Total participants']),
                colorscale='magma',
                cmin=0,
                cmax=1,
                colorbar=dict(thickness=25,
                              len=.9,
                              orientation='h',
                              tickvals=[norm(10),norm(30),norm(50),norm(100),norm(300)],
                              ticktext=['<10',30,50,100,'>300'],
                              ticks="outside",
                              title="Number of participants",
                              titleside="right",
                              tickformatstops=[dict(dtickrange=[0,1])],
                              y=-0.24
                             )
            ),
            x=df_rel.loc[df_rel['Total participants']>=step,'Age'],
            y=df_rel.loc[df_rel['Total participants']>=step,'Percentage female']),
        secondary_y=False
    )

fig.add_trace(
    go.Scatter(visible=False),
    secondary_y=True
)

fig.data[0].visible = True

steps = []
for i in range(len(fig.data)-1):
    step = dict(
        method="update",
        label=i+1,
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Joint distribution of average age of participants and proportion of sex for studies with " + str(i+1) +" or more participants"}],
               # "xaxis" : dict(range=[-5,95],title='Average age (years)')}],
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Minimum number of participants: "},
    pad={"t": 50},
    steps=steps,
    y=-.12,
    len=.95
)]

fig.update_layout(
    sliders=sliders,
    height=800,
    title="Joint distribution of average age of participants and proportion of sex for studies with 1 or more participants",
    xaxis=dict(range=[-5,95],title='Average age (years)'),
    yaxis=dict(title="Percentage of female participants",range=[-6,106]),
    plot_bgcolor='rgba(0.2, 0.2, 0.2, 0.2)',
    width=1100,
)

fig.update_yaxes(title_text="Percentage of male participants", range=[-6,106], tickmode='array', tickvals=[0,20,40,60,80,100], ticktext=[100,80,60,40,20,0], secondary_y=True)

fig.show()

In [ ]:
# x = df_rel['Age']
# y = df_rel['Percentage female']
# w = df_rel['Total participants']
# mx, my = (np.sum(i * w)/np.sum(w) for i in [x,y])
# wpearson = np.sum(w * (x - mx) * (y - my)) / np.sqrt(np.sum(w * (x - mx) * (x - mx)) * np.sum(w * (y - my) * (y - my)))
# wpearson 

# Ethnicity

Here we break down the ethnicity of participants reported in these imaging studies. Box sizes for each category are proportional to their participant counts. Boxes in the treemap may be clicked on to be expanded. Each category's box shows the percentage of all participants in that category and the number of participants. For subcategories within the “reported ethnicity” category, the percentage of the total reported participants is also given.

## Figure 8

In [ ]:
labels = df.columns[8:17].to_list()
parents = ['Non-white','Non-white','Non-white','Reported','Non-white','Non-white','Non-white','Reported','','Reported','']
ethnum = df[df.columns[8:17]].sum()
tot = ethnum.sum()
ethnum['Non-white'] = 0 #df[df.columns[8:15]].sum().sum()
ethnum['Reported'] = 0 #df[df.columns[8:16]].sum().sum()
ethdata = ethnum/tot*100
labels.append('Non-white')
labels.append('Reported')
# labels.append('Total')
# ethdata['Total'] = 0
values = ethdata[labels].to_list()
values = [f'{(round(v,3))}' for v in values]
textbox = ["Percent of total: " + str(x) + "%" for x in values]
textbox[0] = textbox[0] + "<br>Percent of reported: 0.49%<br>Total participants: 12"
textbox[1] = textbox[1] + "<br>Percent of reported: 1.35%<br>Total participants: 33"
textbox[2] = textbox[2] + "<br>Percent of reported: 2.58%<br>Total participants: 63"
textbox[3] = textbox[3] + "<br>Percent of reported: 0.98%<br>Total participants: 24"
textbox[4] = textbox[4] + "<br>Percent of reported: 0.20%<br>Total participants: 5"
textbox[5] = textbox[5] + "<br>Percent of reported: 0.12%<br>Total participants: 3"
textbox[6] = textbox[6] + "<br>Percent of reported: 0.08%<br>Total participants: 2"
textbox[7] = textbox[7] + "<br>Percent of reported: 94.18%<br>Total participants: 2299"
textbox[8] = textbox[8] + "<br>Total participants: 60114"

fig=go.Figure(go.Treemap(
    labels=labels,
    parents=parents,
    values = values,
    hoverinfo='skip',
    text=textbox
))

fig.update_layout(height=500,margin = dict(t=0, l=0, r=50, b=0))
fig.show()

ethdata = df[df.columns[8:17]].sum()

# Trends in participant sex by year of publication

Based on the age distribution, it appears that many healthy participant studies likely recruit from student university volunteer populations. We wanted to explore whether this, or other factors, might influence the sex distribution of participants over time, given the significant improvements in gender representation within scientific fields in recent decades. Additionally it is valuable to visualize the total number of participants included in imaging studies over time. The stacked bar chart shows the number of male and female participants by year of publication, with the full height of each bar representing the total participants studied in that year. The red line overlaid on the graph indicates the percentage of female participants in studies each year.

## Figure 9

In [ ]:
# grouping by year of publication and summing male and female participants
df_grouped = df_full[['Year of Publication','Male','Female']].copy().fillna(0)
df_grouped['Male'] = pd.to_numeric(df_grouped['Male'], errors='coerce').fillna(0)
df_grouped['Female'] = pd.to_numeric(df_grouped['Female'], errors='coerce').fillna(0)
df_ageyear = df_grouped.groupby('Year of Publication').sum()
df_ageyear = df_ageyear.loc[df_ageyear.index[0:32]]
s1 = df_ageyear.loc[:,['Female','Male']].sum(axis=1)# / df_ageyear.loc[:,['Female','Male']].sum(axis=1) *100
s2 = df_ageyear['Female'] #/ df_ageyear.loc[:,['Female','Male']].sum(axis=1) *100
s3 = df_ageyear['Female'] / df_ageyear.loc[:,['Female','Male']].sum(axis=1) *100
df_ageyear['Male'] = s1
df_ageyear['Female'] = s2


ax1 = sns.set_style(style=None, rc=None)
fig, ax1 = plt.subplots(figsize=(12,6))

b1 = sns.barplot(x='Year of Publication', y='Male', data=df_ageyear, color='darkblue', ax=ax1)
b2 = sns.barplot(x='Year of Publication', y='Female', data=df_ageyear, color='lightblue', ax=ax1)
top = mpatches.Patch(color='darkblue', label='Male participants')
bottom = mpatches.Patch(color='lightblue', label='Female participants')
plt.xticks(rotation=-60)
plt.ylabel("Number of participants")
plt.title("Participant sex in imaging studies by year of publication")

df_ageyear['Percent Female'] = s3

ax2 = plt.twinx()
rl = sns.lineplot(x=np.arange(32), y='Percent Female', ax=ax2, marker='o', color='firebrick', data=df_ageyear, label='Percent female',legend=False)

ax2.set_ylabel('Percent female')
ax2.set_ylim(0,100)
ax2.set_yticks([0,10,20,30,40,50,60,70,80,90,100])
ax2.grid()

handle, label = ax2.get_legend_handles_labels()
plt.legend(handles=[top,bottom,handle[0]],loc='upper left')
plt.show()

# Age distribution by region

Since we know that the majority of MRI and PET studies take place in Montreal, it is worth investigating whether location influences the age distribution of participants. Given that fewer studies are conducted in other regions, namely Sherbrooke and Quebec City, these sites may be more selective in their participant recruitment or focused on studying more specific populations rather than enrolling any available healthy volunteers. 

The following raincloud plot visualizes this distribution. The box plots display the median (centerline), first and third quartiles (box edges), and the minimum and maximum data points at the edge of the whiskers (known as fences), excluding any outliers. In the scatter plots below each boxplot, each black dot represents a study. The overlaid half-violin plots illustrate the density of studies across the average age range for each region. Each distribution can be hovered over to display summary statistics.

## Figure 10

In [ ]:
filtage = filtered[['Geographical location','Average Age (Years)']].copy().dropna()
filtage['Average Age (Years)'] = pd.to_numeric(filtage['Average Age (Years)'], errors='coerce').dropna()
filtage = filtage.dropna()

# fig,ax = plt.subplots(figsize=(9,5))
# bp_colors = ['olive','turquoise','orchid']
# bp_data = [filtage.loc[filtage['Geographical location'] == 'Montreal','Average Age (Years)'],
#            filtage.loc[filtage['Geographical location'] == 'Sherbrooke','Average Age (Years)'],
#            filtage.loc[filtage['Geographical location'].str.contains('Quebec'),'Average Age (Years)']]

colors = plotly.colors.PLOTLY_SCALES["Viridis"]

fig = go.Figure()

for i in range(0,3):
    fig.add_trace(go.Violin(y=filtage['Geographical location'][(filtage['Geographical location'] == pd.unique(filtage['Geographical location'])[i])],
                            x=filtage['Average Age (Years)'][(filtage['Geographical location'] == pd.unique(filtage['Geographical location'])[i])],
                            side='positive',
                            pointpos=-.3,
                            line_color='black',
                            orientation='h',
                            fillcolor=colors[i*4+4][1],
                            marker=dict(size=3),
                            spanmode='hard',
                            box=dict(width=.5),
                            # hoverinfo='none',
                            showlegend=False,
                            width=.9)
             )

fig.update_traces(box_visible=True,
                  points='all', 
                  jitter=0.1,  
                  scalemode='count')
fig.update_layout(
    title_text="Average age of studies by region",
    height=500,
    width=900,
    xaxis=dict(title='Average age (years)'),
    plot_bgcolor='rgba(0.2, 0.2, 0.2, 0.2)',
    violingap=0, violingroupgap=0, violinmode='overlay')
fig.show()
# flierprops = dict(marker='.', markerfacecolor='black', markersize=4,
#                   markeredgecolor='none')

# bp = ax.boxplot(bp_data, patch_artist=True, vert=False, flierprops=flierprops, medianprops=dict(color='grey'))

# for patch, color in zip(bp['boxes'], bp_colors):
#     patch.set_facecolor(color)
#     patch.set_alpha(.1)

# vp = ax.violinplot(bp_data, points=500, showmeans=False,
#                    showextrema=False, vert=False)

# for i, b in enumerate(vp['bodies']):
#     m = np.mean(b.get_paths()[0].vertices[:,0])
#     b.get_paths()[0].vertices[:,1] = np.clip(b.get_paths()[0].vertices[:,1], i+1, i+2)
#     b.set_color(bp_colors[i])

# for i, feature in enumerate(bp_data):
#     y = np.full(len(feature), i+.8)
#     idxs = np.arange(len(y))
#     out = y.astype(float)
#     out.flat[idxs] += np.random.uniform(low=-.05, high=.05, size=len(idxs))
#     y=out
#     plt.scatter(feature, y, s=.3, c=bp_colors[i])

# # plt.title('Distribution of average age of studies according to region')
# ax.set_yticklabels(['Montreal','Sherbrooke','Quebec City'])
# plt.xlabel('Average age of participants (Years)')
# plt.xticks(np.arange(0,100,10))
# plt.show()

In [ ]:
# statistical test not to be included in final pub
# from scikit_posthocs import posthoc_dunn
# import scipy as sp
# bp_data = [filtage.loc[filtage['Geographical location'] == 'Montreal','Average Age (Years)'],
#            filtage.loc[filtage['Geographical location'] == 'Sherbrooke','Average Age (Years)'],
#            filtage.loc[filtage['Geographical location'].str.contains('Quebec'),'Average Age (Years)']]
# a = sp.stats.shapiro(bp_data[0].to_list())
# b = sp.stats.kruskal(bp_data[0].to_list(),bp_data[1].to_list(),bp_data[2].to_list())
# c = posthoc_dunn([bp_data[0].to_list(),bp_data[1].to_list(),bp_data[2].to_list()])
# print(b)
# print(c)

# Age distribution by study population category

We are particularly interested in how the average age distribution varies across different population categories, especially those expected to be skewed, such as Alzheimer's. The overall age distribution shown in Figure 4 can be seen as a composition of these individual distributions. By separating them, we gain further insight into how different study populations are selected. 

The following raincloud plot visualizes these distributions, similar to Figure 10, but categorized by study population. The box plots show the median (center line), first and third quartiles (box edges), and the minimum and maximum data points at the edge of the whiskers (known as fences), excluding any outliers. In the scatter plots below each boxplot, each black dot represents a study. The overlaid half-violin plots illustrate the density of studies across average participant age for each category. Each distribution can be hovered over to display summary statistics.


## Figure 11

In [ ]:
df_paths2 = df_paths.loc[df_paths['Average age'].notna()]

colors = plotly.colors.PLOTLY_SCALES["Viridis"]

fig = go.Figure()

for i in range(0,len(pd.unique(df_paths2['Pathology']))):
    fig.add_trace(go.Violin(y=df_paths2['Pathology'][(df_paths2['Pathology'] == pd.unique(df_paths2['Pathology'])[i])],
                            x=df_paths2['Average age'][(df_paths2['Pathology'] == pd.unique(df_paths2['Pathology'])[i])],
                            side='positive',
                            pointpos=-.3,
                            line_color='black',
                            orientation='h',
                            fillcolor=colors[i][1],
                            marker=dict(size=3),
                            spanmode='hard',
                            box=dict(width=.5),
                            # hoverinfo='none',
                            showlegend=False)
             )

fig.update_traces(box_visible=True,
                  points='all', 
                  jitter=0.1,  
                  scalemode='count')
fig.update_layout(
    title_text="Average age of studies by population category",
    height=1100, width=1100,
    xaxis=dict(title='Average age (years)'),
    plot_bgcolor='rgba(0.2, 0.2, 0.2, 0.2)',
    violingap=0, violingroupgap=0, violinmode='overlay')
fig.show()

# Trends in demographic reporting

Given our findings on the lack of demographic reporting, we wanted to examine whether this trend has changed over time. This final visualization tracks the reporting of demographic information over time. A bar plot in the background represents the total number of participants per year, while three line plots show the percentage of studies reporting each demographic variable over time.

## Figure 12

In [ ]:
df_full['Unreported sex'] = df_full.loc[:,'Total participants'] - df_full.loc[:,['Male','Female']].sum(axis=1)
df_full['Reported sex'] = df_full.loc[:,['Male','Female']].sum(axis=1)

df_full.loc[df_full['Average age'].notna(),'Unreported age'] = 0
df_full.loc[df_full['Average age'].isna(),'Unreported age'] = df_full.loc[df_full['Average age'].isna(),'Total participants']
df_full.loc[df_full['Average age'].notna(),'Reported age'] = df_full.loc[df_full['Average age'].notna(),'Total participants']
df_full.loc[df_full['Average age'].isna(),'Reported age'] = 0

df_full.loc[df_full['Ethnicity']!='Ethnic distribution not specified','Unreported ethnicity'] = 0
df_full.loc[df_full['Ethnicity']!='Ethnic distribution not specified','Reported ethnicity'] = df_full.loc[df_full['Ethnicity']!='Ethnic distribution not specified','Total participants']
df_full.loc[df_full['Ethnicity']=='Ethnic distribution not specified','Unreported ethnicity'] = df_full.loc[df_full['Ethnicity']=='Ethnic distribution not specified','Total participants']
df_full.loc[df_full['Ethnicity']=='Ethnic distribution not specified','Reported ethnicity'] = 0


df_grouped = df_full[['Year of Publication','Unreported sex','Reported sex','Unreported age','Reported age','Unreported ethnicity','Reported ethnicity','Total participants']].copy().fillna(0)
dfrep = df_grouped.groupby('Year of Publication').sum()
dfrep = dfrep.loc[dfrep.index[0:32]]
s1 = dfrep['Total participants']
s2 = dfrep['Reported sex'] / dfrep['Total participants'] * 100
s3 = dfrep['Reported age'] / dfrep['Total participants'] * 100
s4 = dfrep['Reported ethnicity'] / dfrep['Total participants'] * 100

plt.style.use('ggplot')
ax1 = sns.set_style(style=None, rc=None)
fig, ax1 = plt.subplots(figsize=(12,6))


sns.set_palette("colorblind")

b1 = sns.barplot(x='Year of Publication', y='Total participants', data=dfrep, color='darkblue', ax=ax1)
# b2 = sns.barplot(x='Year of Publication', y='Unreported sex', data=df_sexrep, color='lightblue', ax=ax1)
top = mpatches.Patch(color='darkblue', label='Total participants')
# bottom = mpatches.Patch(color='lightblue', label='Unreported sex')
plt.xticks(rotation=-60)
plt.ylabel("Number of participants")
ax1.grid(False)
# plt.title("Demographic reporting in imaging studies by year of publication")

dfrep['Percent sex reported'] = s2
dfrep['Percent age reported'] = s3
dfrep['Percent ethnicity reported'] = s4

ax2 = plt.twinx()
rl = sns.lineplot(x=np.arange(32), y='Percent sex reported', ax=ax2, marker='o', data=dfrep,  label='Percent sex reported',legend=False)
r2 = sns.lineplot(x=np.arange(32), y='Percent age reported', ax=ax2, marker='o', data=dfrep,  label='Percent age reported',legend=False)
r3 = sns.lineplot(x=np.arange(32), y='Percent ethnicity reported', ax=ax2, marker='o', data=dfrep,  label='Percent ethnicity reported',legend=False)

ax2.set_ylabel('Percent reported')
ax2.set_ylim(0,100)
ax2.set_yticks([0,10,20,30,40,50,60,70,80,90,100])
ax2.grid(True)

handle, label = ax2.get_legend_handles_labels()
plt.legend(handles=[top,handle[0],handle[1],handle[2]],loc='center left',bbox_to_anchor=(0,.35))
plt.show()